# Agrupación y Segmentación de Vecindarios en la Ciudad de Toronto, Canadá

#### Instalacion de librerias

In [1]:
#pip install geopy

In [2]:
#pip install beautifulsoup4

In [3]:
#pip install geocoder

In [4]:
#pip install folium

In [5]:
#conda install -c conda-forge folium

In [6]:
import numpy as np # librería para manejar datos vectorizados

import pandas as pd # librería para análisis de datos
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # librería para manejar archivos JSON 

#!conda install -c conda-forge geopy --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare 
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas

# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

## Inciso 1

Proceso ETL de informacion de WIKIPEDIA a Dataframe

#### Carga de HTML desde WIKIPEDIA

In [7]:
from bs4 import BeautifulSoup
import requests

#Extraccion
url_wiki="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url_wiki)
bs = BeautifulSoup(response.text, 'html.parser')
contenedor=bs.find(id="mw-content-text").div.table.tbody

In [8]:
#crear data frame en blanco
#Codigo postal, ciudad, y vecindario
nombre_columnas =['Codigo_postal','Ciudad','Vecindario']
PD_vecindarios=pd.DataFrame(columns=nombre_columnas)

#### Proceso de lectura de HTML a Dataframe

In [9]:
#Primer ETL Python

#Transformacion
for fila_table in contenedor.find_all('tr'):
    #print (fila_table)
    #print ('FILA==================================================================')
    for columna_table in fila_table.find_all('td'):
        #print (columna_table)
        #print(columna_table.p)
        ciudad_cruda=columna_table.p.span
        ciudad_barrio=ciudad_cruda.find_all('a')
        #print(ciudad_barrio)
        vecindarios =""
        ciudad=""
        codigo_postal=""
        codigo_postal=columna_table.p.b.get_text()
        cargar=0
        for index,datos_columna in enumerate(ciudad_barrio):
            cnt_registros=len(ciudad_barrio)
            #print (cnt_registros,'---',index)
            if index ==0:
                #print ('Captura Ciudad')
                ciudad=datos_columna.get_text()
                if cnt_registros==1:
                    #separar por parentesis
                    txt_split=ciudad_cruda.get_text().replace(")","").split("(")
                    ciudad=txt_split[0]
                    vecindarios=txt_split[1]
            else:
                if cnt_registros>1:
                    vecindarios=vecindarios+', '+datos_columna.get_text()

        if(vecindarios[:2])==", ":
            vecindarios=vecindarios[2:]
        #print(len(ciudad_barrio),"-------",len(ciudad),"--",'Codigo postal: [',codigo_postal,']',"Ciudad: [",ciudad,"]",'Vecindarios : [',vecindarios,']') 
        #Carga
        if len(ciudad_barrio)>0:
            PD_vecindarios = PD_vecindarios.append({'Codigo_postal':codigo_postal,
                                          'Ciudad':ciudad,
                                          'Vecindario':vecindarios}, ignore_index=True)      
print('Datos cargados..')        

Datos cargados..


##### Presentacion de datos

In [10]:
print (PD_vecindarios.shape)
PD_vecindarios

(101, 3)


,Codigo_postal,Ciudad,Vecindario
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Inciso 2

Utilizar el paquete Geocoder o el archivo csv para crear un dataframe con las coordenadas de cada vecindario.

Metodo implementado, a raiz de no poder realizar la implementacion de GeoCoder por falta de respuesta de la API, fue realizar la __carga por CSV__.

In [11]:
url_csv="http://cocl.us/Geospatial_data";
csv_url='https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv'


#Presentacion de datos de CSV
df_ll = pd.read_csv(csv_url)
df_ll.shape

(103, 3)

In [12]:
#Merge de datos por codigo postal
df_final=PD_vecindarios
df_final = df_final.join(df_ll.set_index('Postal Code'), on='Codigo_postal')
df_final.head()

,Codigo_postal,Ciudad,Vecindario,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [13]:
print(df_final.shape)

(101, 5)


In [14]:
print('El dataframe tiene {} ciudades y {} vecindarios.'.format(
        len(df_final['Ciudad'].unique()),
        df_final.shape[0]
    )
)

El dataframe tiene 23 ciudades y 101 vecindarios.


# Inciso 3

{Proceso de exploracion y agrupacion de vecindarios de __Toronto__, se analizaran unicamente las ciudades que contengan la palabra Toronto en su nombre.

In [15]:
df_final.dtypes

Codigo_postal     object
Ciudad            object
Vecindario        object
Latitude         float64
Longitude        float64
dtype: object

Transformacion de campos de Ciudad y vecindario para su posterior analisis

In [16]:
df_final["Ciudad"]=df_final["Ciudad"].astype('string')
df_final["Vecindario"]=df_final["Vecindario"].astype('string')

#### Filtrando Ciudades que incluyan la palabra Toronto

In [17]:
toronto_data=df_final.query('Ciudad.str.contains("Toronto")', engine='python').reset_index(drop=True)

#Presentacion de datos
print (toronto_data.shape)
toronto_data.groupby('Ciudad').count()

(25, 5)


,Codigo_postal,Vecindario,Latitude,Longitude
Ciudad,,,,
Central Toronto,5,5,5,5
Downtown Toronto,17,17,17,17
Downtown TorontoStn A PO Boxes25 The Esplanade,1,1,1,1
East Toronto,1,1,1,1
East TorontoBusiness reply mail Processing Centre969 Eastern,1,1,1,1


Obtenemos la latitus y longitud de Toronto, Ontario.

In [18]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


Generacion de mapa con los vecindarios por encima.

In [19]:
# crear un mapa de Toronto Otario utilizando los valores de latitud y longitud
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Ciudad'], toronto_data['Vecindario']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

__Definicion de credenciales de Foursquare__

In [20]:
# The code was removed by Watson Studio for sharing.

Credenciales cargadas..


Funcion para obtener la __categoria, nombre, latitud y longitud__ de los vecindarios analizados por medio de la API de Foursquare.

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # solicitud GET
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # regresa solo información relevante de cada sitio cercano
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Vecindario', 
                  'Vecindario Latitude', 
                  'Vecindario Longitude', 
                  'Lugar', 
                  'Lugar Latitude', 
                  'Lugar Longitude', 
                  'Categoria Lugar']
    
    return(nearby_venues)

Proceso de depuracion de Dataframe

In [22]:
toronto_venues = getNearbyVenues(names=toronto_data['Vecindario'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Bay Street
Christie
Richmond, King
Harbourfront, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Lawrence Park
Davisville North
Forest Hill North & West
North Toronto West
Davisville
University of Toronto
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
Enclave of M4L


In [23]:
#Presentacion de datos.
toronto_venues.head()

,Vecindario,Vecindario Latitude,Vecindario Longitude,Lugar,Lugar Latitude,Lugar Longitude,Categoria Lugar
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Gilead Café & Bistro,43.653578,-79.360884,New American Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [24]:
#Conteo de vecindarios
toronto_venues.groupby('Vecindario').count()

,Vecindario Latitude,Vecindario Longitude,Lugar,Lugar Latitude,Lugar Longitude,Categoria Lugar
Vecindario,,,,,,
Bay Street,64,64,64,64,64,64
Berczy Park,59,59,59,59,59,59
"CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport",15,15,15,15,15,15
Christie,14,14,14,14,14,14
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37
Davisville North,7,7,7,7,7,7
Enclave of M4L,17,17,17,17,17,17


In [25]:
print('Tenemos {} categorias unicas.'.format(len(toronto_venues['Categoria Lugar'].unique())))

Tenemos 217 categorias unicas.


#### Proceso de Codificacion

In [26]:
# codificación
toronto_onehot = pd.get_dummies(toronto_venues[['Categoria Lugar']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
toronto_onehot['Vecindario'] = toronto_venues['Vecindario'] 

# mover la columna de barrio a la primer columna
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Vecindario,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [27]:
toronto_onehot.shape

(1299, 218)

In [28]:
toronto_grouped = toronto_onehot.groupby('Vecindario').mean().reset_index()
toronto_grouped.head()

,Vecindario,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.015625,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.015625,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.015625,0.000000,0.0,0.000000,0.0,0.015625,0.0,0.031250,0.000000,0.0,0.0,0.062500,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.203125,0.0,0.0,0.0,0.0,0.000000,0.015625,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.031250,0.015625,0.015625,0.015625,0.0,0.000000,0.0,0.015625,0.0,0.000000,0.0,0.000000,0.0,0.015625,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.015625,0.000000,0.015625,0.0,0.0,0.0,0.0,0.015625,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.015625,0.0,0.0,0.015625,0.015625,0.0,0.0,0.03125,0.031250,0.000000,0.0,0.015625,0.015625,0.

Tamaño de dataframe codificado

In [29]:
toronto_grouped.shape

(25, 218)

Visualizar el top 5 de lugares por cada Vecindario

In [30]:
num_top_venues = 5

for hood in toronto_grouped['Vecindario']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Vecindario'] == hood].T.reset_index()
    temp.columns = ['Lugar','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bay Street----
              Lugar  freq
0       Coffee Shop  0.20
1              Café  0.06
2    Sandwich Place  0.05
3      Burger Joint  0.03
4  Department Store  0.03


----Berczy Park----
            Lugar  freq
0     Coffee Shop  0.07
1  Farmers Market  0.03
2        Pharmacy  0.03
3             Pub  0.03
4      Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport----
              Lugar  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3   Harbor / Marina  0.07
4               Bar  0.07


----Christie----
           Lugar  freq
0  Grocery Store  0.29
1           Café  0.21
2           Park  0.14
3     Baby Store  0.07
4    Candy Store  0.07


----Church and Wellesley----
                 Lugar  freq
0  Japanese Restaurant  0.08
1          Coffee Shop  0.06
2     Sushi Restaurant  0.06
3           Restaurant  0.04
4              Gay Bar  0.04


----Commerce Court, Victoria Hotel----
   

###### Ordenar lugares de forma descendente para el analisis de top 10

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Vecindario']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Lugar mas comun'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Lugar mas comun'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Vecindario'] = toronto_grouped['Vecindario']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Vecindario,1st Lugar mas comun,2nd Lugar mas comun,3rd Lugar mas comun,4th Lugar mas comun,5th Lugar mas comun,6th Lugar mas comun,7th Lugar mas comun,8th Lugar mas comun,9th Lugar mas comun,10th Lugar mas comun
0,Bay Street,Coffee Shop,Café,Sandwich Place,Burger Joint,Department Store,Salad Place,Japanese Restaurant,Italian Restaurant,Spa,Park
1,Berczy Park,Coffee Shop,Farmers Market,Pharmacy,Pub,Restaurant,Beer Bar,Bakery,Cocktail Bar,Seafood Restaurant,Cheese Shop
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Airport
3,Christie,Grocery Store,Café,Park,Baby Store,Candy Store,Nightclub,Coffee Shop,Restaurant,Movie Theater,Music Venue
4,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Gay Bar,Café,Bubble Tea Shop,Hotel,Mediterranean Restaurant,Men's Store


### Definicion de modelo __K-means__

In [33]:
# establecer el número de agrupaciones
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Vecindario', 1)

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

##### Data frame con los sitios mas populares de cada vecindario

In [34]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Etiquetas de cluster', kmeans.labels_)

toronto_merged = toronto_data

# juntar manhattan_grouped con manhattan_data 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Vecindario'), on='Vecindario')

toronto_merged.head() # revisar las ultimas columnas

,Codigo_postal,Ciudad,Vecindario,Latitude,Longitude,Etiquetas de cluster,1st Lugar mas comun,2nd Lugar mas comun,3rd Lugar mas comun,4th Lugar mas comun,5th Lugar mas comun,6th Lugar mas comun,7th Lugar mas comun,8th Lugar mas comun,9th Lugar mas comun,10th Lugar mas comun
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Park,Pub,Theater,Dessert Shop,Breakfast Spot,Brewery,Shoe Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Lingerie Store,Thai Restaurant,Electronics Store,Hotel
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Hotel,Clothing Store,Cocktail Bar,Gastropub,Restaurant,Seafood Restaurant,Department Store,Creperie
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Health Food Store,Pub,Neighborhood,Trail,Asian Restaurant,Organic Grocery,Optical Shop,Mediterranean Restaurant,Men's Store,Mexican Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Farmers Market,Pharmacy,Pub,Restaurant,Beer Bar,Bakery,Cocktail Bar,Seafood Restaurant,Cheese Shop


## Creacion de mapa con las agrupaciones resultantes del modelo.

In [35]:
# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Vecindario'], toronto_merged['Etiquetas de cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Agrupaciones
### Etiqueta 1

In [36]:
toronto_merged.loc[toronto_merged['Etiquetas de cluster'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Ciudad,Etiquetas de cluster,1st Lugar mas comun,2nd Lugar mas comun,3rd Lugar mas comun,4th Lugar mas comun,5th Lugar mas comun,6th Lugar mas comun,7th Lugar mas comun,8th Lugar mas comun,9th Lugar mas comun,10th Lugar mas comun
0,Downtown Toronto,0,Coffee Shop,Café,Bakery,Park,Pub,Theater,Dessert Shop,Breakfast Spot,Brewery,Shoe Store
1,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Lingerie Store,Thai Restaurant,Electronics Store,Hotel
2,Downtown Toronto,0,Coffee Shop,Café,Hotel,Clothing Store,Cocktail Bar,Gastropub,Restaurant,Seafood Restaurant,Department Store,Creperie
4,Downtown Toronto,0,Coffee Shop,Farmers Market,Pharmacy,Pub,Restaurant,Beer Bar,Bakery,Cocktail Bar,Seafood Restaurant,Cheese Shop
5,Downtown Toronto,0,Coffee Shop,Café,Sandwich Place,Burger Joint,Department Store,Salad Place,Japanese Restaurant,Italian Restaurant,Spa,Park
7,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Hotel,Restaurant,Gym,Deli / Bodega,Thai Restaurant,Cosmetics Shop,Pizza Place
8,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Pizza Place,Sporting Goods Shop
9,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Salad Place,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Steakhouse
10,Downtown Toronto,0,Coffee Shop,Restaurant,Hotel,Café,Gym,Italian Restaurant,Deli / Bodega,Cocktail Bar,Seafood Restaurant,Japanese Restaurant
12,Central Toronto,0,Park,Gym,Food & Drink Shop,Hotel,Breakfast Spot,Department Store,Sandwich Place,Adult Boutique,Museum,Middle Eastern Restaurant


### Etiqueta 2

In [37]:
toronto_merged.loc[toronto_merged['Etiquetas de cluster'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Ciudad,Etiquetas de cluster,1st Lugar mas comun,2nd Lugar mas comun,3rd Lugar mas comun,4th Lugar mas comun,5th Lugar mas comun,6th Lugar mas comun,7th Lugar mas comun,8th Lugar mas comun,9th Lugar mas comun,10th Lugar mas comun
11,Central Toronto,1,Bus Line,Park,Swim School,Adult Boutique,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
13,Central Toronto,1,Park,Trail,Jewelry Store,Sushi Restaurant,Bus Line,Adult Boutique,Movie Theater,Men's Store,Mexican Restaurant,Middle Eastern Restaurant


### Etiqueta 3

In [38]:
toronto_merged.loc[toronto_merged['Etiquetas de cluster'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Ciudad,Etiquetas de cluster,1st Lugar mas comun,2nd Lugar mas comun,3rd Lugar mas comun,4th Lugar mas comun,5th Lugar mas comun,6th Lugar mas comun,7th Lugar mas comun,8th Lugar mas comun,9th Lugar mas comun,10th Lugar mas comun
6,Downtown Toronto,2,Grocery Store,Café,Park,Baby Store,Candy Store,Nightclub,Coffee Shop,Restaurant,Movie Theater,Music Venue


### Etiqueta 4

In [39]:
toronto_merged.loc[toronto_merged['Etiquetas de cluster'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Ciudad,Etiquetas de cluster,1st Lugar mas comun,2nd Lugar mas comun,3rd Lugar mas comun,4th Lugar mas comun,5th Lugar mas comun,6th Lugar mas comun,7th Lugar mas comun,8th Lugar mas comun,9th Lugar mas comun,10th Lugar mas comun
3,East Toronto,3,Health Food Store,Pub,Neighborhood,Trail,Asian Restaurant,Organic Grocery,Optical Shop,Mediterranean Restaurant,Men's Store,Mexican Restaurant


### Etiqueta 5

In [40]:
toronto_merged.loc[toronto_merged['Etiquetas de cluster'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Ciudad,Etiquetas de cluster,1st Lugar mas comun,2nd Lugar mas comun,3rd Lugar mas comun,4th Lugar mas comun,5th Lugar mas comun,6th Lugar mas comun,7th Lugar mas comun,8th Lugar mas comun,9th Lugar mas comun,10th Lugar mas comun
19,Downtown Toronto,4,Park,Playground,Trail,Adult Boutique,Museum,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
